### Data cleaning steps for file

RowNumber: The sequential number assigned to each row in the dataset.

CustomerId: A unique identifier for each customer.

Surname: The surname of the customer.

CreditScore: The credit score of the customer.

Geography: The geographical location of the customer (e.g., country or region).

Gender: The gender of the customer.

Age: The age of the customer.

Tenure: The number of years the customer has been with the bank.

Balance: The account balance of the customer.

NumOfProducts: The number of bank products the customer has.

HasCrCard: Indicates whether the customer has a credit card (binary: yes/no).

IsActiveMember: Indicates whether the customer is an active member (binary: yes/no).

EstimatedSalary: The estimated salary of the customer.

Exited: Indicates whether the customer has exited the bank (binary: yes/no).

In [172]:
import pandas as pd
df = pd.read_csv("../data/raw/Churn_Modelling.csv")
print(df.head(n=1))

   RowNumber  CustomerId   Surname  CreditScore Geography  Gender  Age  \
0          1    15634602  Hargrave          619    France  Female   42   

   Tenure  Balance  NumOfProducts  HasCrCard  IsActiveMember  EstimatedSalary  \
0       2      0.0              1          1               1        101348.88   

   Exited  
0       1  


In [173]:
# Remove duplicates
df = df.drop_duplicates()

# Convert categorical to lowercase
cat_cols = df.select_dtypes(include="object").columns
df[cat_cols] = df[cat_cols].apply(lambda col:col.str.lower())

In [152]:
# Check for outlliers (credit score, salary, balance)
df.describe()

,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.00000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,5000.50000,1.569094e+07,650.528800,38.921800,5.012800,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,2886.89568,7.193619e+04,96.653299,10.487806,2.892174,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,1.00000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,2500.75000,1.562853e+07,584.000000,32.000000,3.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,5000.50000,1.569074e+07,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,7500.25000,1.575323e+07,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,10000.00000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

numeric_cols = df.select_dtypes(include=['int64','float64']).columns

for col in numeric_cols:
    plt.figure(figsize=(6,3))
    sns.boxplot(x=df[col])
    plt.title(col)
    plt.show()

In [42]:
# Checking the NumOfProducts outlier
above_4 = df[df["NumOfProducts"] == 4]["NumOfProducts"].count()
print(above_4)

60


### Handling any missing values

In [174]:
# Median for salaries/balance
num_cols = ["Balance", "EstimatedSalary"]
for col in num_cols:
    df[col] = df[col].fillna(df[col].median())

In [175]:
# Mode for geography
for col in cat_cols:
    df[col] = df[col].fillna(df[col].mode()[0])

In [177]:
# Drop rows missing many field values
threshold = df.shape[1]/2 # Missing half the values
df = df.dropna(thresh=threshold)

### Create new features for modeling

In [178]:
import numpy as np

In [179]:
# Balance-to-salary ratio
df['BalanceSalaryRatio'] = np.log1p(df['Balance'] / df['EstimatedSalary'])

In [180]:
# Create interaction feature to see if certain regions with low credit score are more at risk.
df["GeographyCreditInteraction"] = df["Geography"] + '_' + pd.cut(df["CreditScore"], bins=[300,600,700,850], labels=["low", "medium", "high"]).astype(str)

In [181]:
# Customers with high tenure AND many products may be less likely to churn.
df['TenureProducts'] = df['Tenure'] * df['NumOfProducts']

In [182]:
# Combine activity and balance into one feature
df['ActiveHighBalance'] = df['IsActiveMember'] * (df['Balance'] > df['Balance'].median()).astype(int)
df['ActiveLowBalance'] = df['IsActiveMember'] * (df['Balance'] < df['Balance'].median()).astype(int)

A Risk Score Bucket is a way to group customers based on their risk of churn or financial risk into discrete categories. In your dataset, you can create it using a combination of features like CreditScore, Balance, Age, NumOfProducts, or even model-predicted probabilities later.

In [183]:
# Bucket credit scores into risk levels: low, medium, high.
df['RiskScoreBucket'] = pd.cut(df['CreditScore'], bins=[300, 580, 700, 850],labels=['high risk', 'medium risk', 'low risk'])

A Salary Bracket is a way to group customers based on their EstimatedSalary into categories, instead of using raw numeric values. This helps the model capture patterns like “low-salary customers behave differently than high-salary customers” without being affected by outliers or skewed distributions.

In [184]:
# Salary bracket
bins = [0, 50000, 100000, 150000, 200000]
labels = ['very low', 'low', 'medium', 'high']
df['SalaryBracket'] = pd.cut(df['EstimatedSalary'], bins=bins, labels=labels)

In [185]:
# Age groups (for stage effect)
df["AgeGroup"] = pd.cut(df["Age"], bins=[18,30,45,60,100], labels=["18-30", "31-45", "46-60", "60+"])

# Drop rows where AgeGroup is NaN
df = df.dropna(subset=['AgeGroup'])

In [186]:
# Some banks find younger males or older females more likely to churn.
df['GenderAgeGroup'] = df['Gender'] + '_' + df['AgeGroup'].astype(str)


In [187]:
# Flag customers with above-median balance.
df["IsHighBalance"] = (df["Balance"] > df["Balance"].median()).astype(int)

In [188]:
# Flag customers with multiple products: single, multiple, maxed out.
df["ProductsLevel"] = pd.cut(df["NumOfProducts"], bins=[0,1,2,5], labels=["single", "few", "many"])

In [189]:
# Tenure buckets
bins = [0, 2, 5, 10, df["Tenure"].max()+1]
labels = ['0-2 years', '3-5 years', '6-10 years', '10+ years']
df['tenure_bucket'] = pd.cut(df['Tenure'], bins=bins, labels=labels, right=True)

In [190]:
print(df.columns)

Index(['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited', 'BalanceSalaryRatio',
       'GeographyCreditInteraction', 'TenureProducts', 'ActiveHighBalance',
       'ActiveLowBalance', 'RiskScoreBucket', 'SalaryBracket', 'AgeGroup',
       'GenderAgeGroup', 'IsHighBalance', 'ProductsLevel', 'tenure_bucket'],
      dtype='object')


In [191]:
# Convert categorical to lowercase
cat_cols = df.select_dtypes(include="object").columns
df[cat_cols] = df[cat_cols].apply(lambda col:col.str.lower())

In [192]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,...,TenureProducts,ActiveHighBalance,ActiveLowBalance,RiskScoreBucket,SalaryBracket,AgeGroup,GenderAgeGroup,IsHighBalance,ProductsLevel,tenure_bucket
0,1,15634602,hargrave,619,france,female,42,2,0.00,1,...,2,0,1,medium risk,medium,31-45,female_31-45,0,single,0-2 years
1,2,15647311,hill,608,spain,female,41,1,83807.86,1,...,1,0,1,medium risk,medium,31-45,female_31-45,0,single,0-2 years
2,3,15619304,onio,502,france,female,42,8,159660.80,3,...,24,0,0,high risk,medium,31-45,female_31-45,1,many,6-10 years
3,4,15701354,boni,699,france,female,39,1,0.00,2,...,2,0,0,medium risk,low,31-45,female_31-45,0,few,0-2 years
4,5,15737888,mitchell,850,spain,female,43,2,125510.82,1,...,2,1,0,low risk,low,31-45,female_31-45,1,single,0-2 years


In [193]:
df.to_csv("../data/clean/Churn_Modeling.csv", index=False)